# **Reciprocal space and Brillouin zone**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/band-theory/brillouin_zone.ipynb
<p style="text-align: justify;font-size:15px">
    In this notebook we explore two central concepts in electronic structure and band theory: those of reciprocal space and the Brillouin zone.
</p>
<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**
* Appreciate the nature of the relationship between the direct and reciprocal space descriptions of a crystalline system.
* Use this understanding to explain the differences in the Brillouin zones observed for different crystal structures. 


<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [1]:
from widget_bzvisualizer import BZVisualizer
import numpy as np
import seekpath
import nglview as nv
from ase.build import bulk, molecule
from ipywidgets import HBox, VBox, Button, Output, Text, Tab, Layout, Label, HTML, Dropdown
import ipywidgets as widgets
import nglview as nv
from ase import Atom, Atoms
from ase.symbols import symbols2numbers
from ase.data import atomic_numbers
from ase.lattice.cubic import BodyCenteredCubic,SimpleCubic, FaceCenteredCubic
from ase.lattice.orthorhombic import *
from ase.lattice.triclinic import Triclinic
from ase.lattice.monoclinic import *
from ase.lattice.tetragonal import *


# from ase.lattice.orthorhombic import FaceCenteredOrthorhombic,BaseCenteredOrthorhombic
from ase.lattice.hexagonal import Hexagonal

In [2]:
bt_compute = Button(description="Update")

# mat = bulk('Cu', 'sc', a=3.6)
# mat = mat.repeat([3, 3, 3])

# mat = SimpleCubic(symbol='Cu',latticeconstant=3.6)
mat = SimpleOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': 3.6, 'b': 3.6, 'c': 3.6})


m = nv.NGLWidget(width='400px', height='400px')
m.background='black'
c1 = m.add_component(nv.ASEStructure(mat))

m.clear()
m.add_unitcell()
m.add_ball_and_stick()
mpos = [];
msym = [];

for i in mat:
    print(i)
    mpos.append(i.position.tolist())
    msym.append('Cu')
print(mat)

a=3.6
b=3.6
c=3.6
real_lattice=np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) *a/ 2.0;

w = BZVisualizer(real_lattice, [[0.0, 0.0, 0.0]], [1], True, height='400px',width='400px')
#w.cell = (np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]) * a / 2.0).tolist()
# cella = (mat[1].position - mat[0].position).tolist()
# cellb = (mat[2].position - mat[0].position).tolist()
# cellc = (mat[3].position - mat[0].position).tolist()

# w = BZVisualizer([cella, cellb, cellc], [mat[0].position.tolist()], [58], True, height='400px', width='400px')

Atom('Cu', [0.0, 0.0, 0.0], index=0)
Lattice(symbols='Cu', pbc=True, cell=[3.6, 3.6, 3.6])


In [ ]:
# w = BZVisualizer(mat.cell.tolist(), mpos, symbols2numbers(mat.get_chemical_symbols()), 
#                  True, height='400px', width='400px',enable_interaction=True)



"""
now contents are dropdwn menus. 
Depending on the crystal family selected, there shall be a dropdown menu 

"""
crystal_family = 'Cubic'
crystal_family_dropdown = widgets.Dropdown(options=['Cubic','Orthorhombic','Tetragonal','Triclinic','Monoclinic','Hexagonal','Rhombohedral'],
                                           value='Cubic',
                                           description='Crystal family:',
                                           disabled=False)

centering_dropdown = widgets.Dropdown(options=['Primitive','Body-centered', 'Face-centered'],
                                           value='Primitive',
                                           description='Crystal family:',
                                           disabled=False)

tab_contents = ['Crystal family', 'Centering'] 
# children = [widgets.Text(description=name) for name in tab_contents]
children = [crystal_family_dropdown, centering_dropdown]

tab = widgets.Tab()
tab.children = children

# tab.set_title(0, 'Crystal family')
# tab.set_title(1, 'b')
# tab.set_title(2, 'c')

tab.set_title(0, 'Crystal family')
tab.set_title(1, 'Centering')

# tab.children[0].value = str(mat.cell[0].tolist())
# tab.children[1].value = str(mat.cell[1].tolist())
# tab.children[2].value = str(mat.cell[2].tolist())


output = Output(layout=Layout(width="400px"))





 
tab2 = Tab()

a_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='a:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

b_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='b:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

c_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='c:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

alpha_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\alpha$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

beta_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\beta$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

gamma_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\gamma$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


def fix_sliders_ab(a):
    # fix value of b to that of a
    return a

def fix_sliders_ac(a):
    # fix value of b and c sliders to that of a
    return a

def fix_sliders_alpha_beta(alpha):
    # fix value of b and c sliders to that of a
    return alpha

def fix_sliders_alpha_gamma(alpha):
    # fix value of b and c sliders to that of a
    return alpha

# set up link between a and b sliders
l_ab = widgets.dlink((a_slider, 'value'), (b_slider, 'value'),transform=fix_sliders_ab)

# set up link between a and c sliders
l_ac = widgets.dlink((a_slider, 'value'), (c_slider, 'value'),transform=fix_sliders_ac)

# set up link between alpha and beta sliders
l_alpha_beta = widgets.dlink((alpha_slider, 'value'), (beta_slider, 'value'),transform=fix_sliders_alpha_beta)
l_alpha_beta.unlink()

# set up link between alpha and gamma sliders
l_alpha_gamma = widgets.dlink((alpha_slider, 'value'), (gamma_slider, 'value'),transform=fix_sliders_alpha_gamma)
l_alpha_gamma.unlink()

# default settings for cubic crystal type
alpha_slider.value=90
beta_slider.value=90
gamma_slider.value=90
alpha_slider.disabled=True
beta_slider.disabled=True
gamma_slider.disabled=True
l_ab.link()
b_slider.disabled=True
l_ac.link()
c_slider.disabled=True


tab2.children = [VBox([a_slider,b_slider,c_slider]), VBox([alpha_slider,beta_slider,gamma_slider])]

# tab2.children = [VBox([HBox([atom_num, atom_pos]), HBox([bt_add, bt_clear])]), 
#                  VBox([HBox([ed_sym, ed_pos]), HBox([ed_index, ed_get, ed_set])]), output]

tab2.set_title(0, 'Lattice vectors')
tab2.set_title(1, 'Axial angles')
tab2.set_title(2, 'List of atoms')

debug_output =widgets.Output(layout={'border': '1px solid black'})

display(debug_output)

# Function to initialize crystal structure set up lattice vector and axial angle selection according to choice 
# of crystal family + centering
def reset_sliders(change):
    # crystal_family = str(tab.children[0].value)
    
    
   # get also the value of the centering chosen by the user before updating the crystal structure     
    
    
    global crystal_family_dropdown,crystal_family
    if change['type'] == 'change' and change['name'] == 'value':

        crystal_family = str(change.new)

        with debug_output:
                print("crystal_family=="+str(crystal_family))
       

        if(crystal_family=='Cubic'):
              
            # lock b and c sliders to that of a
            # lock angles to 90,90,90
            centering_dropdown.options=['Primitive','Body-centered', 'Face-centered']
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            b_slider.disabled=True
            l_ac.link()
            c_slider.disabled=True
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()
            
            
            

        elif(crystal_family=='Orthorhombic'):
            with debug_output:
                print("crystal_family=="+str(crystal_family))  
            # vector lengths are all independent
            # lock angles to 90,90,90
            centering_dropdown.options=['Primitive','Base-centered','Body-centered', 'Face-centered']

            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.unlink()
            l_ac.unlink()
            b_slider.disabled=False
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()



        elif(crystal_family=='Tetragonal'):
            with debug_output:
                print("crystal_family=="+str(crystal_family))
                
            centering_dropdown.options=['Primitive','Body-centered']
            # lock b slider value to that of a
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            b_slider.disabled=True
            l_ac.unlink()
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()
            
            


        elif(crystal_family=='Monoclinic'):
            # two lattice vectors meet at 90 degrees.
            # The third meets at an angle beta
            # all 3 lattice vectors can take unique values
            
            centering_dropdown.options=['Primitive','Base-centered']
            alpha_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.unlink()
            l_ac.unlink()
            b_slider.disabled=False
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()
            
            

            
        elif(crystal_family=='Hexagonal'):
            # a=b with 120 degrees between
            # a and c are perpendicular. Similarly for b and c
            # all 3 lattice vectors can take unique values
          
            centering_dropdown.options=['Primitive']
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=120
            alpha_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            l_ac.unlink()
            b_slider.disabled=True
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()
            
            


        elif(crystal_family=='Rhombohedral'):
            # a=b=c. Only need to specify a and alpha.
            centering_dropdown.options=['Primitive']
            alpha_slider.disabled=False
            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            l_ac.link()
            b_slider.disabled=True
            c_slider.disabled=True
            l_alpha_beta.link()
            l_alpha_gamma.link()
            
            

        elif(crystal_family=='Triclinic'):
            # all 3 lattice vectors and angles may vary independently
            
            centering_dropdown.options=['Primitive']
            alpha_slider.disabled=False
            beta_slider.disabled=False
            gamma_slider.disabled=False
            l_ab.unlink()
            l_ac.unlink()
            b_slider.disabled=False
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()
            
            

        
            
crystal_family_dropdown.observe(reset_sliders)


def compute_BZ(c):
    global m,c1, crystal_family,debug_output,w, real_lattice
    # get mag of lattice vec from slider and construct vector
    a = a_slider.value
    b = b_slider.value
    c = c_slider.value
    
    alpha=alpha_slider.value
    beta=beta_slider.value
    gamma=gamma_slider.value
    
    centering=centering_dropdown.value
    
    mpos = [];
    msym = [];
    with debug_output:
        print("a="+str(a))
        print("centering="+centering)

    if(crystal_family=='Cubic'):
        if(centering=='Primitive'):
            
            real_lattice  =  np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) *a / 2.0
            mat = SimpleOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})

        elif(centering=='Body-centered'):
            real_lattice = np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]) *a / 2.0;
            mat = BodyCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})

        elif(centering=='Face-centered'):
            real_lattice=np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]) *a/ 2.0;
            mat = FaceCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})                         
            
    if(crystal_family=='Orthorhombic'):
        if(centering=='Primitive'):
            real_lattice  =  np.array([[a, 0, 0], [0, b, 0], [0, 0, c]])
            mat = SimpleOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})

        elif(centering=='Base-centered'):
            with debug_output:
                print("base-centered ortho")
            real_lattice  =  np.array([[a, -b, 0], [a, b, 0], [0, 0, 2*c]]) / 2.0
            #real_lattice=np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]) *a/ 2.0;
            mat = BaseCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
        elif(centering=='Body-centered'):
            real_lattice  =  np.array([[-a, b, c], [a, -b, c], [a, b, -c]])/2
            mat = BodyCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
        elif(centering=='Face-centered'):
            real_lattice  =  np.array([[0, b, c], [a, 0, c], [a, b, 0]])/2
            mat = FaceCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
    if(crystal_family=='Triclinic'):
            
            c_x = c*np.cos(np.pi*(beta/180))
            c_y = (1./np.sin(np.pi*(90/180)))*c*(np.cos(np.pi*(alpha/180))-np.cos(np.pi*(beta/180))*np.cos(np.pi*(gamma/180)))
            c_z = np.sqrt(c**2-c_x**2-c_y**2)
            real_lattice  =  np.array([[a, 0, 0], [b*np.cos(np.pi*(gamma/180)), b*np.sin(np.pi*(gamma/180)), 0], [c_x, c_y, c_z]])               
            mat = Triclinic(
                              size=(1,1,1), symbol='Cu', 
                              latticeconstant={'a': a, 'b': b, 'c': c,'alpha':alpha,'beta':beta,'gamma':gamma})

    if(crystal_family=='Monoclinic'):
        if(centering=='Primitive'):
            real_lattice=np.array([[a,0,0], [0, b, 0], [c*np.cos(np.pi*(beta/180)),0,c*np.sin(np.pi*(beta/180))]])
            mat = SimpleMonoclinic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c,'alpha':alpha})
        elif(centering=='Base-centered'):
            real_lattice=np.array([[a/2,-b/2,0], [a/2, b/2, 0], [c*np.cos(np.pi*(beta/180)),0,c*np.sin(np.pi*(beta/180))]])
            mat = BaseCenteredMonoclinic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c,'alpha':alpha})
    if(crystal_family=='Tetragonal'):
        if(centering=='Primitive'):
            real_lattice=np.array([[a,0,0], [0, a, 0], [0,0,c]])
            mat = SimpleTetragonal(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
        elif(centering=='Body-centered'):
            real_lattice=np.array([[-a,a,c], [a, -a, c], [a,a,-c]])/2
            mat = BodyCenteredTetragonal(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(1,1,1), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
    if(crystal_family=='Rhombohedral'):
            mat = bulk('Cu', 'rhombohedral', a=a,alpha=alpha,basis=[[0,0,0],[0,0,0],[0,0,0]])
            c_z=np.sqrt((1/3.)*(4*np.cos(np.pi*(alpha/2/180))**2-1))
            # rhombohedral/trigonal lattices are not implemented in ase's lattice module
            #real_lattice= np.array([[a/2, -a/(2*np.sqrt(3)), c/3], [0,a/np.sqrt(3), c/3],[-a/2, -a/(2*np.sqrt(3)), c/3]]) 
            real_lattice= a*np.array([[np.sin(np.pi*(alpha/2/180)), -1/np.sqrt(3)*np.sin(np.pi*(alpha/(2*180))), c_z], [0,2/np.sqrt(3)*np.sin(np.pi*(alpha/(2*180))), c_z],[-np.sin(np.pi*(alpha/2/180)), -1/(np.sqrt(3))*np.sin(np.pi*(alpha/2/180)), c_z]]) 

    if(crystal_family=='Hexagonal'):
        mat = Hexagonal(size=(1,1,1), symbol='Cu',latticeconstant={'a':a,'c':c})
        real_lattice= np.array([[a, 0, 0], [a/2, np.sqrt(3)/2*a, c],[0, 0, c],]) 


    w.cell = real_lattice.tolist()
  
    m.remove_component(c1)
    m.clear()

    with debug_output:
            print(mat)
    c1 = m.add_component(nv.ASEStructure(mat))
    m.clear()
    # cella = (mat1[1].position - mat1[0].position).tolist()
    # cellb = (mat1[2].position - mat1[0].position).tolist()
    # cellc = (mat1[3].position - mat1[0].position).tolist()

    # for i in mat:
    #     with debug_output:
    #         print("positions")
    #         print(i)
    #     mpos.append(i.position.tolist())
    #     msym.append('Cu')
        

#     v1 = list(eval(tab.children[0].value))
#     v2 = list(eval(tab.children[1].value))
#     v3 = list(eval(tab.children[2].value))
    
#     assert type(v1) == list
#     assert type(v2) == list
#     assert type(v3) == list
        
    # logic for initialization of crystal structure in direct space    
        
    # mat.set_cell([cella, cellb, cellc])
    # mat = BodyCenteredCubic(directions=[[1,0,0], [0,1,0], [1,1,1]],
    #                       size=(2,2,3), symbol='Cu', pbc=(1,1,0),
    #                       latticeconstant=4.0)
    
   
    # w = BZVisualizer(mat.cell.tolist(), mpos, symbols2numbers(mat.get_chemical_symbols()), 
    m.add_ball_and_stick()
    m.add_unitcell()
    

    
    # w.cell = [cella, cellb, cellc]
    # w.positions = [mat1[0].position.tolist()]
    # with debug_output:
    #     print('BZ')
    #     print(cella)
    #     print(w.positions)
    #w = BZVisualizer(mat.cell.tolist(), , symbols2numbers(mat.get_chemical_symbols()), 
                 #True, height='400px', width='400px',enable_interaction=True)
    # w = BZVisualizer([cella, cellb, cellc], [mat[0].position.tolist()], [58], True, height='400px', width='400px')
    # w = BZVisualizer([cella, cellb, cellc], mpos, symbols2numbers(mat.get_chemical_symbols()), 
    #              True, height='400px', width='400px',enable_interaction=True)
    
   
    # with debug_output:
    #     print(w)             
    # w.numbers = symbols2numbers(mat.get_chemical_symbols())
    # w.positions = [mat[0].position.tolist()]
    #w.cell = mat1.cell.tolist()
    
    # w.cell = mat.cell.tolist()
    #w.update_structure += 1
    


bt_compute.on_click(compute_BZ);

label1 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='blue'><font size=5>Structure</b></div>")
label2 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='blue'><font size=5>Brillouin zone</b></div>")

display(HBox([label1, label2]), HBox([m, w]), HBox([tab, tab2]), bt_compute)